In [1]:
from __future__ import print_function 
import os, sys
import numpy as np
from six.moves import cPickle
from collections import OrderedDict

import tensorflow as tf
sys.path.append('..')
from tfomics import neuralnetwork as nn
from tfomics import neuralbuild as nb
from tfomics import utils, learn

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

X_train = mnist.train.images
X_train = np.reshape(X_train, (X_train.shape[0], 28, 28, 1))
y_train = mnist.train.labels

X_valid = mnist.validation.images
X_valid = np.reshape(X_valid, (X_valid.shape[0], 28, 28, 1))
y_valid = mnist.validation.labels

X_test = mnist.test.images
X_test = np.reshape(X_test, (X_test.shape[0], 28, 28, 1))
y_test = mnist.test.labels

# get shapes
num_data, height, widht, dim = X_train.shape
input_shape=[None, height, widht, dim]
num_labels = y_train.shape[1]   # number of labels (output units)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:

# create model
layer1 = {'layer': 'input',
        'input_shape': input_shape,
        'name': 'input'
        }
layer2 = {'layer': 'conv2d', 
        'num_filters': 32,
        'filter_size': (5, 5),
        #'norm': 'batch',
        'activation': 'prelu',
        'dropout': 0.1,
        'name': 'conv1'
        }
layer3 = {'layer': 'conv2d', 
        'num_filters': 64,
        'filter_size': (5, 5),
        #'norm': 'batch',
        'activation': 'prelu',
        'dropout': 0.1,
        'name': 'conv2'
        }
layer4 = {'layer': 'dense', 
        'num_units': 512,
        'activation': 'prelu',
        'dropout': 0.5,
        'name': 'dense1'
        }  
layer5 = {'layer': 'dense', 
        'num_units': num_labels,
        'activation': 'softmax',
        'name': 'dense2'
        }

#from tfomics import build_network
model_layers = [layer1, layer2, layer3, layer4, layer5]

# optimization parameters
optimization = {"objective": "categorical",
              "optimizer": "adam",
              "learning_rate": 0.001,      
              "l2": 1e-6,
              }

In [4]:
nnbuild = nb.NeuralBuild(model_layers)
network, placeholders, hidden_feed_dict = nnbuild.get_network_build()

In [5]:
# build neural network class
nnmodel = nn.NeuralNet(network, placeholders, hidden_feed_dict)
nnmodel.inspect_layers()

----------------------------------------------------------------------------
Network architecture:
----------------------------------------------------------------------------
layer1: input
(?, 28, 28, 1)
layer2: conv1
(?, 24, 24, 32)
layer3: conv1_bias
(?, 24, 24, 32)
layer4: conv1_active
(?, 24, 24, 32)
layer5: conv1_dropout
(?, 24, 24, 32)
layer6: conv2
(?, 20, 20, 64)
layer7: conv2_bias
(?, 20, 20, 64)
layer8: conv2_active
(?, 20, 20, 64)
layer9: conv2_dropout
(?, 20, 20, 64)
layer10: dense1
(?, 512)
layer11: dense1_bias
(?, 512)
layer12: dense1_active
(?, 512)
layer13: dense1_dropout
(?, 512)
layer14: dense2
(?, 10)
layer15: dense2_bias
(?, 10)
layer16: output
(?, 10)
----------------------------------------------------------------------------


In [6]:
# set output file paths
data_path = '/home/peter/Code/tensorflow/data'
filename = 'MNIST'
save_path = utils.make_directory(data_path, filename)
filepath = os.path.join(save_path, filename)

# compile neural trainer
nntrainer = nn.NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)

In [7]:
train = {'inputs': [X_train], 'targets': y_train}
valid = {'inputs': [X_valid], 'targets': y_valid}
data = {'train': train, 'valid': valid}
learn.train_minibatch(nntrainer, data, batch_size=100, num_epochs=50, 
                    patience=10, verbose=2, shuffle=True)

Epoch 1 out of 50 
[==============================] 100.0% -- time=0s -- loss=0.24748 -- accuracy=92.94%   
  valid loss:		0.08788
  valid accuracy:	0.99508+/-0.00150
  valid auc-roc:	0.99927+/-0.00062
  valid auc-pr:		0.99602+/-0.00273
  lower cross-validation found
  saving model to:  /home/peter/Code/tensorflow/data/MNIST/MNIST_best.ckpt
Epoch 2 out of 50 
[====                          ] 12.0% -- time=37s -- loss=0.08760 -- accuracy=97.48%  

KeyboardInterrupt: 